In [23]:
cd /content/drive/My Drive/Colab Notebooks/ProtonX-TransferLearning-Human-Horse

/content/drive/My Drive/Colab Notebooks/ProtonX-TransferLearning-Human-Horse


In [0]:
main_folder = './'
train_folder = './train/'
validation_folder = './validation/'

In [0]:
# !unzip -q horse-or-human.zip -d $train_folder
# !unzip -q validation-horse-or-human.zip -d $validation_folder

In [0]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/content/drive/My Drive/Colab Notebooks/ProtonX-TransferLearning-Human-Horse/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [0]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Flatten, Input, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop

In [0]:
pre_trained_model = InceptionV3(include_top=False, weights = None,
                                input_shape = (200,200,3))
pre_trained_model.load_weights(local_weights_file)

In [0]:
# Data Preprocessing

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(rescale = 1.0/255.0, 
                                    rotation_range= 0.2, zoom_range = 0.1,
                                    # samplewise_center=True, samplewise_std_normalization=True,
                                    width_shift_range = 0.2, height_shift_range = 0.2,
                                    horizontal_flip = True)

valid_generator = ImageDataGenerator(rescale = 1.0/255.0,
      # samplewise_center=True, samplewise_std_normalization=True,
      )

In [0]:
size_image = (200,200)
batch_size = 32

In [31]:
train_generator_data = train_generator.flow_from_directory(
    train_folder, target_size = size_image , batch_size = batch_size, shuffle = True,
    class_mode = 'binary')

valid_generator_data = valid_generator.flow_from_directory(
    validation_folder, target_size = size_image, batch_size = batch_size,
    class_mode = 'binary')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [32]:
train_generator_data[0][0]

array([[[[0.56995225, 0.49130008, 0.43348935],
         [0.57008106, 0.49140745, 0.43356806],
         [0.5702099 , 0.49151483, 0.43364683],
         ...,
         [0.20727615, 0.22012612, 0.19977583],
         [0.26793793, 0.27696177, 0.22427401],
         [0.30361405, 0.35328418, 0.35657266]],

        [[0.56674874, 0.4843958 , 0.42165068],
         [0.56667715, 0.4843242 , 0.4215791 ],
         [0.56660557, 0.48425263, 0.42150754],
         ...,
         [0.2881074 , 0.3215278 , 0.31769243],
         [0.36326045, 0.37562796, 0.2878886 ],
         [0.38960296, 0.41347927, 0.33578578]],

        [[0.49788707, 0.4107468 , 0.3432144 ],
         [0.49772245, 0.41056788, 0.34302115],
         [0.49755782, 0.41038892, 0.3428279 ],
         ...,
         [0.37346652, 0.3782014 , 0.25259614],
         [0.4087569 , 0.41672927, 0.2853387 ],
         [0.43538398, 0.44495913, 0.30928674]],

        ...,

        [[0.5568628 , 0.48235297, 0.41176474],
         [0.5568628 , 0.48235297, 0.41176474]

In [33]:
train_generator_data.class_indices
valid_generator_data.class_indices

{'horses': 0, 'humans': 1}

In [0]:
pre_trained_model.trainable = True

In [35]:
len(pre_trained_model.trainable_variables)

188

In [36]:
len(pre_trained_model.layers)

311

In [0]:
for layer in pre_trained_model.layers[:280]:
  layer.trainable = False # Only use last layers from layer 260th

In [38]:
len(pre_trained_model.trainable_variables)

18

In [0]:
Fl = Flatten()
Den = Dense(1024, activation = 'relu')
Drop = Dropout(0.2)
# global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
Out = Dense(1, activation = 'sigmoid')

In [0]:
model = tf.keras.Sequential([pre_trained_model,
                             Fl, Den, Drop, Out])

In [0]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [42]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 4, 4, 2048)        21802784  
_________________________________________________________________
flatten_1 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              33555456  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 1025      
Total params: 55,359,265
Trainable params: 39,630,017
Non-trainable params: 15,729,248
_________________________________________________________________


In [0]:
DESIRED_ACCURACY = 0.9950

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_accuracy')>DESIRED_ACCURACY):
      print("\nReached 99.5% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

In [44]:
model.fit(train_generator_data, epochs = 10,
          steps_per_epoch = 100,
          callbacks = [callbacks], 
          validation_data = valid_generator_data,
          validation_steps = 100,
          verbose = 1)

Epoch 1/10
100/100 [==============================] - 61s 611ms/step - loss: 0.5102 - accuracy: 0.9762 - val_loss: 0.5200 - val_accuracy: 0.9762
Epoch 2/10
100/100 [==============================] - 60s 596ms/step - loss: 0.5010 - accuracy: 0.9942 - val_loss: 0.5104 - val_accuracy: 0.9884
Epoch 3/10
100/100 [==============================] - 59s 587ms/step - loss: 0.5004 - accuracy: 0.9974 - val_loss: 0.5150 - val_accuracy: 0.9800
Epoch 4/10
100/100 [==============================] - 58s 580ms/step - loss: 0.4998 - accuracy: 0.9971 - val_loss: 0.5102 - val_accuracy: 0.9884
Epoch 5/10
100/100 [==============================] - ETA: 0s - loss: 0.4996 - accuracy: 0.9994
Reached 99.5% accuracy so cancelling training!
100/100 [==============================] - 58s 581ms/step - loss: 0.4996 - accuracy: 0.9994 - val_loss: 0.5038 - val_accuracy: 1.0000


In [0]:
model.save('human_horse.h5')